In [50]:
import nltk
#import spacy
import pandas as pd
from tqdm import tqdm
import os
import nltk
import numpy as np
from pattern.text.en import singularize
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
import pickle
from collections import Counter
from scipy.stats import chi2_contingency

In [51]:
#get all entities
ent_df = pd.read_csv("Litcovid_treatment_ents.csv")

#create multiindex df to sort by pmid
tuples = list(zip(*[ent_df['pmid'], ent_df['Unnamed: 0']]))
index = pd.MultiIndex.from_tuples(tuples, names=['pmid', 'orig_index'])
pmid_ent_df = ent_df.copy()
pmid_ent_df.drop(["Unnamed: 0"], axis=1, inplace=True)
pmid_ent_df.index = index

In [52]:
pmid_ent_df.head(10)

text         type  negation  \
pmid     orig_index                                                
31981224 0                       covid-19  Participant         0   
         1                       relevant  Participant         0   
         2           host defense against      Outcome         0   
         3                       covid-19  Participant         0   
31996494 4                       covid-19  Participant         0   
32013309 5                   inflammation      Outcome         0   
         6                       covid-19  Participant         0   
         7                      inhibited     modifier         0   
32015560 8                     accelerate     modifier         0   
         9                        control      Outcome         0   

                                                              sentence  \
pmid     orig_index                                                      
31981224 0           In this review , we provide an update on CoV i...   
         1           In this review , we provide an update on CoV i...   
         2           In this review , we provide an update on CoV i...   
         3           This highlights the importance of immune respo...   
31996494 4           As of January 22 , 2020 , a total of 571 cases...   
32013309 5           Here , we propose for the first time that infl...   
         6           Here , we propose for the first time that infl...   
         7           Here , we propose for the first time that infl...   
32015560 8                                    accelerate its control .   
         9                                    accelerate its control .   

                         section      pmid  
pmid     orig_index                         
31981224 0            BACKGROUND  31981224  
         1            BACKGROUND  31981224  
         2            BACKGROUND  31981224  
         3           CONCLUSIONS  31981224  
31996494 4                 TITLE  31996494  
32013309 5               RESULTS  32013309  
         6               RESULTS  32013309  
         7               RESULTS  32013309  
32015560 8            BACKGROUND  32015560  
         9            BACKGROUND  32015560

In [53]:
print(set(pmid_ent_df['type']))
print(set(pmid_ent_df['section']))

{'modifier', 'Intervention', 'Outcome', 'measure', 'Participant'}
{'CONCLUSIONS', 'TITLE', 'OBJECTIVE', 'METHODS', 'RESULTS', 'BACKGROUND', 'ERROR'}


In [6]:
all_pmids = list(set(ent_df.pmid))
num_total_pmid = len(all_pmids)
num_total_pmid

6002

In [7]:
#get all pmids with hcq as intervention: text = hydroxychloroquine or HCQ or hcq
hcq_pmids = ent_df[(ent_df.text.isin(["hydroxychloroquine", 'HCQ', 'hcq'])) & (ent_df.type == "Intervention")].pmid.unique()

#get entities with hcq pmids
hcq_ent_df = pmid_ent_df.loc[hcq_pmids]
hcq_ent_df

text          type  negation  \
pmid     orig_index                                                           
32075365 312                     COVID-19 pneumonia   Participant         0   
         313                     hydroxychloroquine  Intervention         0   
         314                                improve      modifier         0   
         315              success rate of treatment       Outcome         0   
         316                                shorten      modifier         0   
         317                          hospital stay       Outcome         0   
         318                                improve      modifier         0   
         319                        patient outcome       Outcome         0   
         320                     hydroxychloroquine  Intervention         0   
         321                     COVID-19 pneumonia   Participant         0   
         322                     Guangdong Province  Intervention         0   
         323                     hydroxychloroquine  Intervention         0   
         324                     COVID-19 pneumonia   Participant         0   
         325           chloroquine phosphate tablet  Intervention         0   
         326                500mg twice per day for       measure         0   
         327         , moderate and severe cases of   Participant         0   
         328                     COVID-19 pneumonia   Participant         0   
         329                     hydroxychloroquine  Intervention         0   
32147496 1275                              covid-19       Outcome         0   
         1276                    hydroxychloroquine  Intervention         0   
         1277                               inhibit      modifier         0   
         1278                SARS Cov-2 replication       Outcome         0   
32150618 1347                    hydroxychloroquine  Intervention         0   
         1348                       EC50 = 0.72 muM       measure         0   
         1349                           more potent      modifier         0   
         1350                    hydroxychloroquine  Intervention         0   
         1351                       EC50 = 5.47 muM       measure         0   
         1352                    400 mg twice daily       measure         0   
         1353            hydroxychloroquine sulfate  Intervention         0   
         1354                                200 mg       measure         0   
...                                             ...           ...       ...   
32777550 121554                       pediatric IBD   Participant         0   
         121555                            covid-19   Participant         0   
         121556                                 IBD   Participant         0   
         121557                            covid-19       Outcome         0   
         121558                    Crohn 's disease   Participant         0   
         121559                  ulcerative colitis   Participant         0   
         121560                                  49       measure         0   
         121561               indeterminate colitis   Participant         0   
         121562                                  42       measure         0   
         121563                        missing data       Outcome         0   
         121564               cumulative prevalence       Outcome         0   
         121565                               0.4 %       measure         0   
         121566                     Viral infection   Participant         0   
         121567                     more frequently      modifier         0   
         121568                                male   Participant         0   
         121569                              female   Participant         0   
         121570                              56.5 %       measure         0   
         121571                              39.7 %       measure         0   
         121572   

In [8]:
diab_df = ent_df[(ent_df.text == "diabetes") & (ent_df.type == "Participant")]
diab_pmids = diab_df.pmid.unique()

In [9]:
preg_df = ent_df[(ent_df.text == "pregnant") & (ent_df.type == "Participant")]
preg_pmids = preg_df.pmid.unique()

In [10]:
print(num_total_pmid)
print(len(hcq_pmids))
print(len(diab_pmids))
print(len(preg_pmids))

6002
331
284
21


In [11]:
hcq1_diab1_pmid = list(set(hcq_pmids)&set(diab_pmids))
hcq1_preg1_pmid = list(set(hcq_pmids)&set(preg_pmids))
# print(len(hcq_diab_pmid))
# print(len(hcq_preg_pmid))

NameError: name 'hcq_diab_pmid' is not defined

In [12]:
hcq0_diab1_pmid = list(set(diab_pmids)-set(hcq_pmids))
hcq1_diab0_pmid = list(set(hcq_pmids)-set(diab_pmids))
hcq0_diab0_pmid = list(set(all_pmids)-set(hcq_pmids)-set(diab_pmids))
print(len(hcq0_diab1_pmid))
print(len(hcq1_diab0_pmid))
print(len(hcq0_diab0_pmid))

271
318
5400


In [13]:
# when p is small (eg. <= 0.05), could reject the null hypothesis 
# -> reject the hypothesis that the two categories are independent -> agree that they are dependent
# if p is large, could not reject H0 -> we could not say that the 2 catergories are dependent
obs = np.array([[len(hcq1_diab1_pmid),len(hcq0_diab1_pmid)], [len(hcq1_diab0_pmid), len(hcq0_diab0_pmid)]])
chi2, p, dof, ex = chi2_contingency(obs)
p

0.5647274525647445

In [14]:
p_count = Counter(ent_df[ent_df.type == "Participant"].text.values)
i_count = Counter(ent_df[ent_df.type == "Intervention"].text.values)
o_count = Counter(ent_df[ent_df.type == "Outcome"].text.values)

In [15]:
o_count.most_common(20)

[('dead', 1689),
 ('covid-19', 1064),
 ('outcome', 358),
 ('severe covid-19', 335),
 ('safety', 246),
 ('D-dimer', 211),
 ('fever', 202),
 ('efficacy', 202),
 ('symptom', 199),
 ('comorbidity', 170),
 ('prognosi', 167),
 ('mortality rate', 152),
 ('IL-6', 152),
 ('cough', 144),
 ('severe', 142),
 ('clinical outcomes', 128),
 ('C-reactive protein', 128),
 ('mechanical ventilation', 114),
 ('disease severity', 107),
 ('complication', 104)]

In [21]:
# among all articles that mentioned fever as Outcome
# Diabetes (P) vs HCQ (I) with fever(O)
# p value is large, could not reject H0
k_outcome_df = ent_df[(ent_df.text == "mechanical ventilation") & (ent_df.type == "Outcome")]
k_pmids = k_outcome_df.pmid.unique()

k_hcq_pmids = list(set(hcq_pmids)&set(k_pmids))
k_diab_pmids = list(set(diab_pmids)&set(k_pmids))

k_hcq1_diab1_pmid = list(set(k_hcq_pmids)&set(k_diab_pmids))
k_hcq0_diab1_pmid = list(set(k_diab_pmids)-set(k_hcq_pmids))
k_hcq1_diab0_pmid = list(set(k_hcq_pmids)-set(k_diab_pmids))
k_hcq0_diab0_pmid = list(set(k_pmids)-set(k_hcq_pmids)-set(k_diab_pmids))

print(len(k_hcq1_diab1_pmid))
print(len(k_hcq0_diab1_pmid))
print(len(k_hcq1_diab0_pmid))
print(len(k_hcq0_diab0_pmid))

obs = np.array([[len(k_hcq1_diab1_pmid),len(k_hcq0_diab1_pmid)], [len(k_hcq1_diab0_pmid), len(k_hcq0_diab0_pmid)]])
chi2, p, dof, ex = chi2_contingency(obs)
p

1
11
8
76


0.6913428798699426

In [ ]:
# female(P) vs HCQ (I)
# p-value < 0.1, could reject H0
# With sigificance level 10%, we can say that "female" and "HCQ" are not independent among all the articles on covid-19.
female_df = ent_df[(ent_df.text == "female") & (ent_df.type == "Participant")]
female_pmids = female_df.pmid.unique()

hcq1_female1_pmid = list(set(hcq_pmids)&set(female_pmids))
hcq0_female1_pmid = list(set(female_pmids)-set(hcq_pmids))
hcq1_female0_pmid = list(set(hcq_pmids)-set(female_pmids))
hcq0_female0_pmid = list(set(all_pmids)-set(hcq_pmids)-set(female_pmids))

print(len(hcq1_female1_pmid))
print(len(hcq0_female1_pmid))
print(len(hcq1_female0_pmid))
print(len(hcq0_female0_pmid))

obs = np.array([[len(hcq1_female1_pmid),len(hcq0_female1_pmid)], [len(hcq1_female0_pmid), len(hcq0_female0_pmid)]])
chi2, p, dof, ex = chi2_contingency(obs)
p

- It might not help to limit the scope of the articles with certain outcome if we want to reach the conclusion that 2 terms are not independent to each other.
- It seems like that the indenpendence test should serve as a tool to test the "hypothesis" only after we have one -> to prove that if the independence hypothesis is correct. We still need to search for meaningful pairs from EDA (eg. the popular edges) before testing.

# Kolmogorov-Smirnov statistic on 2 samples

In [26]:
from scipy import stats

In [27]:
ent_df[(ent_df.text.isin(["hydroxychloroquine", 'HCQ', 'hcq'])) & (ent_df.type == "Intervention")]

,Unnamed: 0,text,type,negation,sentence,section,pmid
313,313,hydroxychloroquine,Intervention,0,Here we found that treating the patients diagn...,RESULTS,32075365
320,320,hydroxychloroquine,Intervention,0,In order to guide and regulate the use of chlo...,RESULTS,32075365
323,323,hydroxychloroquine,Intervention,0,and Health Commission of Guangdong Province fo...,CONCLUSIONS,32075365
329,329,hydroxychloroquine,Intervention,0,to chloroquine .,CONCLUSIONS,32075365
1276,1276,hydroxychloroquine,Intervention,0,In vitro data suggest that chloroquine inhibit...,BACKGROUND,32147496
1347,1347,hydroxychloroquine,Intervention,0,RESULTS : Hydroxychloroquine ( EC50 = 0.72 muM...,BACKGROUND,32150618
1350,1350,hydroxychloroquine,Intervention,0,RESULTS : Hydroxychloroquine ( EC50 = 0.72 muM...,BACKGROUND,32150618
1361,1361,hydroxychloroquine,Intervention,0,CONCLUSIONS : Hydroxychloroquine was found to ...,METHODS,32150618
1363,1363,hydroxychloroquine,Intervention,0,CONCLUSIONS : Hydroxychloroquine was found to ...,METHODS,32150618
1367,1367,hydroxychloroquine,Intervention,0,We propose that the immunomodulatory effect of...,METHODS,32150618


In [28]:
male_df = ent_df[(ent_df.text == "male") & (ent_df.type == "Participant")]
male_pmids = male_df.pmid.unique()

female_df = ent_df[(ent_df.text == "female") & (ent_df.type == "Participant")]
female_pmids = female_df.pmid.unique()

In [29]:
print(len(male_pmids))
print(len(female_pmids))

418
214


In [30]:
hcq1_male1_pmid = list(set(hcq_pmids)&set(male_pmids))
hcq1_female1_pmid = list(set(hcq_pmids)&set(female_pmids))

In [31]:
print(len(hcq1_male1_pmid))
print(len(hcq1_female1_pmid))

29
18


In [32]:
hcq_df = ent_df[(ent_df.text.isin(["hydroxychloroquine", 'HCQ', 'hcq'])) & (ent_df.type == "Intervention")]

In [33]:
# hcq vs diab
# For articles use diabetes as P, the freq of hcq
# count hcq
hcq_in_male_df = hcq_df[hcq_df.pmid.isin(hcq1_male1_pmid)]
hcq_in_male_df.head(3)

,Unnamed: 0,text,type,negation,sentence,section,pmid
15043,15043,hydroxychloroquine,Intervention,0,All hospitalized patients received hydroxychlo...,METHODS,32351040
15266,15266,hydroxychloroquine,Intervention,0,Cardiovascular considerations of therapies cur...,RESULTS,32352535
15267,15267,hydroxychloroquine,Intervention,0,Cardiovascular considerations of therapies cur...,RESULTS,32352535


In [45]:
ent_df[(ent_df.pmid.isin(hcq1_female1_pmid))&(ent_df.type == "Outcome")].text.value_counts()

dead                                 14
symptom                               7
covid-19                              6
fever                                 4
symptomatic                           4
outcome                               4
in-hospital mortality                 4
cough                                 4
ICU admission                         3
cardiomyopathy                        3
efficacy                              3
average recovery time                 3
shortness of breath                   3
alive and out of hospital ''          2
intubation                            2
troponin                              2
pneumonium                            2
sore throat                           2
side effects                          2
median duration                       2
rate of ARDS                          2
hospitalized                          2
septic shock                          2
risk factors                          2
respiratory rate                      2


In [46]:
ent_df[(ent_df.pmid.isin(hcq1_male1_pmid))&(ent_df.type == "Outcome")].text.value_counts()

dead                                                          26
cough                                                          8
in-hospital mortality                                          8
covid-19                                                       8
outcome                                                        7
symptom                                                        7
fever                                                          6
efficacy                                                       4
treatment                                                      4
LPV plasma concentrations                                      4
symptomatic                                                    4
acute kidney injury                                            4
hospitalized                                                   4
D-dimer                                                        4
mechanical ventilation                                         3
average recovery time    

In [35]:
hcq_m_counts = np.array(hcq_in_male_df['pmid'].value_counts())

In [36]:
hcq_in_female_df = hcq_df[hcq_df.pmid.isin(hcq1_female1_pmid)]
hcq_f_counts = np.array(hcq_in_female_df['pmid'].value_counts())

In [37]:
hcq_f_counts

array([6, 5, 3, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64)

In [38]:
# high pvalue, fail to reject H0(the 2 samples are drawn from the same dist)
# -> 'accept' that they are from the same dist 
stats.ks_2samp(hcq_m_counts, hcq_f_counts)

Ks_2sampResult(statistic=0.06130268199233713, pvalue=0.9999999999783917)